In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
s3_prefix = 'Baichuan2-sagemaker'

In [ ]:
!aws s3 cp --recursive data s3://{sagemaker_session_bucket}/{s3_prefix}/data

In [ ]:
from sagemaker.huggingface import HuggingFace

training_input_path = "s3://{}/{}/data".format(sagemaker_session_bucket, s3_prefix)

# hyperparameters, which are passed into the training job
hyperparameters = {
}

In [ ]:
huggingface_estimator = HuggingFace(
    entry_point="fine-tune.sh",
    source_dir="./fine-tune",
    instance_type="ml.g5.2xlarge",
    instance_count=1,
    role=role,
    transformers_version="4.28",
    pytorch_version="2.0",
    py_version="py310",
    hyperparameters=hyperparameters,
)

In [ ]:
huggingface_estimator.fit({"train": training_input_path})

In [ ]:
predictor = huggingface_estimator.deploy(initial_instance_count=1, instance_type="ml.g5.2xlarge")

In [ ]:
input = {
    "inputs": "This is the best movie ever made in history, an absolute sculpted work of art that depicts every emotion of human existence, from suffering, to courage to love, in front of the background of political astuteness and socio-hierarchal analysis."
}

predictor.predict(input)

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()